# Multi agent app

In [292]:
import os

import google.generativeai as genai
from dotenv import load_dotenv
from IPython.display import Markdown
from phi.agent import Agent
from phi.model.google import Gemini
from phi.model.openai import OpenAIChat
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.googlesearch import GoogleSearch
from phi.tools.hackernews import HackerNews
from phi.tools.newspaper4k import Newspaper4k
from phi.tools.yfinance import YFinanceTools

from textwrap import dedent

In [3]:
_ = load_dotenv()

In [224]:
gemini_model = Gemini(api_key=os.getenv("GEMINI_API_KEY"),id="gemini-1.5-flash-latest",
                generation_config=genai.types.GenerationConfig(temperature=0.1,top_p=0.9))

In [221]:
lambda_model = OpenAIChat(id="llama3.1-70b-instruct-berkeley",
                         api_key=os.getenv("LAMBDA_API_KEY"),
                         base_url="https://api.lambdalabs.com/v1",
                         temperature=0.1)

In [222]:
openai_model = OpenAIChat(id="gpt-4o-mini",
          temperature=0.1, api_key=os.getenv("OPENAI_API_KEY")
        )